# Importar Modulos

In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
import plotly.express as px
import plotly.graph_objs as go
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
import tensorflow_addons as tfa
from IPython.display import display
import keras_tuner as kt

C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Importación de Datos

In [3]:
#el conj de datos va de 0 a 2623 o sea 2624 datos
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

# Recompila la información de un archivo CSV y lo guarda en un arreglo
data_copy = np.loadtxt("lp5.csv", delimiter=",", dtype=str)
#  Contador de filas en el CSV
cont = 16
labels = []
numbers = []
# Arreglo de ceros con las dimensiones de los datos
features= np.zeros((164,15,6))

# Ciclo que recorre todos los datos del CSV y guarda los títulos de los datos cuando el contador es 16,
# guarda los números en un arreglo aparte y reestablece el contador a su valor original cuando es igual a 0.
for i in range(len(data_copy)):
    if(cont == 16):
        labels.append(data_copy[i])
    if(cont<16):
        numbers.append(data_copy[i])
    cont -= 1
    if(cont == 0):
        cont = 16
        
# Ciclos anidados que recorren todas las dimensiones del arreglo de ceros,
# agrupa los datos numéricos en conjuntos con forma 15x6 
cont = 0
for i in range(164):
    for j in range(15):
        for z in range(6):
            features[i][j][z] = numbers[cont][z]
        cont += 1
# Convierte los datos obtenidos del primer ciclo en un dataframe
labels_df = pd.DataFrame(labels, columns=['labels','1','2','3','4','5'])
display(labels_df)
display(features)

,labels,1,2,3,4,5
0,normal,,,,,
1,normal,,,,,
2,normal,,,,,
3,normal,,,,,
4,normal,,,,,
...,...,...,...,...,...,...
159,collision_in_tool,,,,,
160,collision_in_tool,,,,,
161,collision_in_tool,,,,,
162,collision_in_tool,,,,,


array([[[ -2.,  -1.,  81.,   0.,  -5.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        ...,
        [ -2.,  -1.,  78.,   0.,  -5.,   0.],
        [ -3.,  -1.,  80.,   1.,  -4.,   1.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.]],

       [[  6.,  -1.,  79.,  -2.,   4.,  -3.],
        [ 42.,  -3.,  80.,   5.,  53.,   3.],
        [ -5.,   4.,  74., -15., -10.,  -1.],
        ...,
        [ -1.,  -5.,  80.,   6.,  -6.,   0.],
        [ -4.,   5.,  78., -14.,  -9.,  -4.],
        [ -4.,   1.,  80.,  -3., -12.,   5.]],

       [[ -2.,  -6.,  85.,  14.,  -5.,   2.],
        [  0.,   2.,  74.,  -7.,   1.,   0.],
        [ -4.,  -5.,  76.,   7., -11.,   4.],
        ...,
        [  0.,  -9.,  87.,  13.,  -5.,   2.],
        [ -5.,   5.,  67., -17., -16.,   7.],
        [ -6., -10.,  86.,  16., -14.,  -1.]],

       ...,

       [[-12.,  17.,   3., -19., -10.,  -4.],
        [-12.,  12.,  11., -13., -16.,  -4.],
        [ -8.,   3.,  

# Manejo de Datos y Normalización

In [4]:
# Diccionario que codifica los datos categóricos a un conjunto de números idóneo para la capa de salida.
classes_dict = {'normal':'1 0 0 0 0', #1
        'collision_in_tool':'0 1 0 0 0', #2
        'collision_in_part':'0 0 1 0 0', #3
        'bottom_collision':'0 0 0 1 0', #4
        'bottom_obstruction':'0 0 0 0 1'} #5

# Se reemplazan los datos por lo del diccionario
labels_df = labels_df.replace({'labels':classes_dict})
# Se separan los datos de manera que se aisle un caractér por columna.
labels_df[['1', '2', '3', '4', '5']] = labels_df['labels'].str.split(' ', 4, expand= True)
# Se eliminan las columnas, solo dejando las columnas llamadas "1", "2", "3", "4", "5"
labels_df = labels_df.loc[:,['1', '2', '3', '4', '5']]
# Se convierten los valores de un tipo String a uno Entero para poder ser utilizados por la red neuronal.
labels_df[['1', '2', '3', '4', '5']]=labels_df[['1', '2', '3', '4', '5']].astype(str).astype(int)

'''/* 
Function: norm

Normaliza los datos a través de una función min-max entre 0 y 1 para ser alimentados a la red.

Parameters:

    x - Datos a normalizar.
    
Returns:

    Los datos alimentados ya normalizados.
*/'''

def norm(x):
    x_min = x.min()
    x_max = x.max()
    range = x_max - x_min  #min max entre 0 y 1
    return((x-x_min)/(range))

# Normaliza las características.
norm_features = norm(features)
# Split para entrenamiento y validacion, con 20% para testeo y se aleatorizan.
train_features, test_features, train_labels, test_labels  = train_test_split(norm_features, labels_df, test_size=0.2, random_state= 42)
display(train_features)
display(train_labels)

C:\Users\dave4\AppData\Local\Temp\ipykernel_14640\2653365139.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  labels_df[['1', '2', '3', '4', '5']] = labels_df['labels'].str.split(' ', 4, expand= True)


array([[[0.77660263, 0.78156702, 0.79084826, 0.77055903, 0.7684006 ,
         0.77422836],
        [0.77466005, 0.78739478, 0.78717893, 0.75156486, 0.7638679 ,
         0.77206993],
        [0.77401252, 0.78804231, 0.79041658, 0.74465789, 0.76106195,
         0.77012735],
        ...,
        [0.78005612, 0.7804878 , 0.78437298, 0.78135118, 0.78113533,
         0.78027196],
        [0.78027196, 0.78091949, 0.77962443, 0.78091949, 0.78027196,
         0.78027196],
        [0.78027196, 0.78070365, 0.78286208, 0.78156702, 0.78091949,
         0.78005612]],

       [[0.78027196, 0.77940859, 0.79905029, 0.78372545, 0.77962443,
         0.78113533],
        [0.78070365, 0.78113533, 0.79667602, 0.77919275, 0.78091949,
         0.78070365],
        [0.77984028, 0.77962443, 0.79710771, 0.78221455, 0.77832938,
         0.78156702],
        ...,
        [0.78070365, 0.77876106, 0.79948198, 0.78350961, 0.77962443,
         0.78113533],
        [0.77962443, 0.78178286, 0.79516512, 0.77703432, 0.777

,1,2,3,4,5
84,0,0,1,0,0
2,1,0,0,0,0
94,0,1,0,0,0
45,0,0,1,0,0
42,0,0,1,0,0
...,...,...,...,...,...
71,0,0,1,0,0
106,0,0,0,1,0
14,1,0,0,0,0
92,0,1,0,0,0


# Creación del Modelo

In [5]:
'''/* 
Function: my_model

Genera el modelo categórico convolucional y lo compila. El modelo consiste de lo siguiente:
    - Una capa de entradas de forma (15,6,1) que es igual a las dimensiones de la matriz de datos.
    
    - Una capa de convolución de dos dimensiones que genera 8 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de convolución de dos dimensiones que genera 8 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de Max Pooling con un pool de 2x2, un salto de 2 casillas y sin padding.
    - Una capa de Dropout razón con una razón de 0.3

    - Una capa de convolución de dos dimensiones que genera 16 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de convolución de dos dimensiones que genera 16 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de Max Pooling con un pool de 2x2, un salto de 2 casillas y sin padding.
    - Una capa de Dropout razón con una razón de 0.2.
    
    - Una capa de flatten para pasar de varias dimensiones a una.
    - Una capa oculta con 24 neuronas y con una función de activación ReLu.
    - Una capa de salida con 5 neuronas. Una por categoría.
    
    - Utiliza ADAM como optimizador.
    - La función de pérdida es la Entropía Cruzada Categórica.
    - La métrica de evaluación es la exactitud.
    
Parameters:

    my_learning_rate - Razón de aprendizaje.
    
Returns:

    El modelo neuronal ya creado.
*/'''

def my_model(learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape = (15,6,1), padding='same'), # #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding= 'valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),# #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding='valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

model = model = my_model(0.01)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 6, 8)          80        
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 6, 8)          584       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 3, 8)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 7, 3, 8)           0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 3, 16)          1168      
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 3, 16)          2320      
                                                        

# Entrenamiento del Modelo

In [6]:
'''/* 
Function: train_model

Alimenta los datos correspondientes a las características y etiquetas al modelo y conduce el proceso de entrenamiento y validación.
Además de esto, también le establece al modelo la cantidad de ciclos de entrenamiento y el tamaño del batch.
Por último crea un subconjunto de validación con un 25% de los datos de entrenamiento.
    
Parameters:

    model - El modelo creado anteriormente.
    features - Conjunto que contiene las características.
    labels - Conjunto que contiene las etiquetas.
    epochs - Cantidad de ciclos de entrenamiento
    batch_size - Tamaño del batch.
    
Returns:

    Los resultados del entrenamiento.
*/'''


def train_model(model, features, labels, epochs, batch_size):
    history = model.fit(
        x = features,
        y = labels,
        epochs= epochs,
        batch_size= batch_size,
        validation_split= 0.25
    )
    # Guarda los resultados obtenidos del proceso de entrenamiento y validación en un DataFrame.
    # Estos resultados son las pérdidas y el número del ciclo correspondiente.
    hist= pd.DataFrame(history.history)
    # Añade al DataFrame la información sobre los ciclos.
    hist['epoch'] = history.epoch
    
    return hist

# Visualizaciones

In [7]:
'''/* 
Function: loss_curve

Grafica las curvas de pérdida correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''

def loss_curves(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de exactitud por una versión más legible
    labels = {"loss":"Training Loss", "val_loss":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Binary Cross Entropy", "variable":"Curvas de Pérdida"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()
    
'''/* 
Function: accuracy_curve

Grafica las curvas de exactitud correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''
    
def accuracy_curve(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de exactitud por una versión más legible
    labels = {"val_accuracy":"Exactitud de Validación", "accuracy":"Exactitud de Entrenamiento"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Exactitud de Entrenamiento', 'Exactitud de Validación'],
                title='Gráficas de Exactitud',
                labels={"epoch": "Epoch", "value":"Exactitud", "variable":"Curvas de Exactitud"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()

# Se corren las funciones

In [8]:
# Hiperparámetros
learning_rate = 0.001
epochs = 300
batch_size = 5
# Llama a la función para crear el modelo y lo guarda.
model = my_model(learning_rate)
# Invoca a la función de entrenamiento y guarda los resultados.
history= train_model(model, train_features, train_labels, epochs, batch_size)
# Llama a la función de las gráficas.
loss_curves(history)
accuracy_curve(history)
display(history)

Epoch 1/300


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 1s 12ms/step - loss: 1.6037 - accuracy: 0.2347 - val_loss: 1.5828 - val_accuracy: 0.2727
Epoch 2/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5887 - accuracy: 0.3163 - val_loss: 1.5707 - val_accuracy: 0.2727
Epoch 3/300
 1/20 [>.............................] - ETA: 0s - loss: 1.5515 - accuracy: 0.4000

C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 0s 4ms/step - loss: 1.5816 - accuracy: 0.3163 - val_loss: 1.5691 - val_accuracy: 0.2727
Epoch 4/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5590 - accuracy: 0.3163 - val_loss: 1.5661 - val_accuracy: 0.2727
Epoch 5/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5437 - accuracy: 0.3163 - val_loss: 1.5744 - val_accuracy: 0.2727
Epoch 6/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5681 - accuracy: 0.3163 - val_loss: 1.5636 - val_accuracy: 0.2727
Epoch 7/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5592 - accuracy: 0.3163 - val_loss: 1.5695 - val_accuracy: 0.2727
Epoch 8/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5552 - accuracy: 0.3163 - val_loss: 1.5680 - val_accuracy: 0.2727
Epoch 9/300
20/20 [==============================] - 0s 4ms/step - loss: 1.5725 - accuracy: 0.2959 - val_loss: 1.5768 - val_accuracy: 0.2727
Epoch 10/300
20/20 [=====

,Training Loss,Exactitud de Entrenamiento,Validation Loss,Exactitud de Validación,epoch
0,1.604,0.235,1.583,0.273,0
1,1.589,0.316,1.571,0.273,1
2,1.582,0.316,1.569,0.273,2
3,1.559,0.316,1.566,0.273,3
4,1.544,0.316,1.574,0.273,4
...,...,...,...,...,...
295,0.866,0.561,0.816,0.697,295
296,0.844,0.541,0.807,0.697,296
297,0.845,0.602,0.838,0.576,297
298,0.843,0.602,0.823,0.667,298


# Predicciones

In [24]:
# Hace predicciones usando el conjunto de datos de prueba.
pd.options.display.float_format = "{:.0f}".format
predictions = model.predict(test_features)
for i in range(len(predictions)):
    if(predictions[i].max() != 1):
        row, col = np.where(predictions == predictions[i].max())
        predictions[i][col[0]] = 1
predictions[predictions!=1] = 0 
    
# Convierte el arreglo de predicciones en un DataFrame
predictions_df = pd.DataFrame(predictions, columns=['normal', 'collision_in_tool', 'collision_in_part', 'bottom_collision', 'bottom_obstruction'])
display(predictions_df)
display(test_labels)
CM = tfa.metrics.MultiLabelConfusionMatrix(num_classes=5)
CM.update_state(test_labels, predictions_df)
result = CM.result().numpy()
display(result)

normal = px.imshow(result[0], 
                labels = dict(x='normal (Predicted)', y='normal'),
                x = ['Negative', '-Positive'], y=['True', 'False'],text_auto=True, color_continuous_scale='greys')
normal.update_xaxes(side="top")
normal.show()

collision_in_tool = px.imshow(result[0], 
                labels = dict(x='collision_in_tool (Predicted)', y='collision_in_tool'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
collision_in_tool.update_xaxes(side="top")
collision_in_tool.show()

collision_in_part = px.imshow(result[0], 
                labels = dict(x='collision_in_part (Predicted)', y='collision_in_part'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
collision_in_part.update_xaxes(side="top")
collision_in_part.show()

bottom_collision = px.imshow(result[0], 
                labels = dict(x='bottom_collision (Predicted)', y='bottom_collision'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
bottom_collision.update_xaxes(side="top")
bottom_collision.show()

bottom_obstruction = px.imshow(result[0], 
                labels = dict(x='bottom_obstruction (Predicted)', y='bottom_obstruction'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
bottom_obstruction.update_xaxes(side="top")
bottom_obstruction.show()
#model.save('ModeloConvolucion_A0.75')

2/2 [==============================] - 0s 3ms/step


,normal,collision_in_tool,collision_in_part,bottom_collision,bottom_obstruction
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,1,0,0,0,0
4,0,0,0,1,0
...,...,...,...,...,...
28,0,0,0,1,0
29,0,0,1,0,0
30,0,0,0,1,0
31,0,0,1,0,0


,1,2,3,4,5
135,0,0,0,1,0
115,0,0,0,0,1
131,0,0,0,0,1
55,1,0,0,0,0
95,0,1,0,0,0
...,...,...,...,...,...
134,0,0,0,1,0
160,0,1,0,0,0
139,0,0,0,1,0
78,0,0,1,0,0


array([[[19.,  3.],
        [ 4.,  7.]],

       [[27.,  0.],
        [ 6.,  0.]],

       [[17.,  9.],
        [ 3.,  4.]],

       [[25.,  3.],
        [ 2.,  3.]],

       [[28.,  1.],
        [ 1.,  3.]]], dtype=float32)